In [2]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
#from google.cloud import storage
#from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time
import torch.nn as nn

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck
import json

#torch.cuda.empty_cache()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


from google.colab import drive
drive.mount('/content/drive/')
os.chdir('drive/MyDrive/Colab Notebooks/AICORE/FAISS')

In [5]:
## Creating a train dataset class
class ItemsTrainDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((225,225))
        #self.rgbify = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)

    def _load_examples(self):
        class_names = os.listdir('pytorch_images_tv/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}
        class_decoder = {idx: class_name for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('pytorch_images_tv/train',cl_name))
            example_fp = [os.path.join('pytorch_images_tv/train', cl_name, img_name ) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)

        features = self.pil_to_tensor(img)
        features = self.resize(features)
        #features = self.rgbify(features)

        return features, img_class

    def __len__(self):
        return len(self.examples)

In [6]:
traindataset = ItemsTrainDataSet()
train_loader = DataLoader(dataset = traindataset, batch_size=64)

In [7]:
def process_img(image):
    pil_to_tensor = transforms.ToTensor()
    resize = transforms.Resize((225,225))
    img = Image.open(image).convert('RGB')

    features = pil_to_tensor(img)
    features = resize(features)
    features = torch.unsqueeze(features, dim=0)
    #print(features.shape)
    return features

In [8]:
## Created a classifier based on the RESNET50 pretrained model

class ItemFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,1000)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [11]:
model = ItemFeatureExtractor()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
checkpoint = torch.load('final_weights/weights.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']


Using cache found in /home/ubuntu/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


## remove last layer to 2048 feature output
model = nn.Sequential(*list(model.resnet50.children())[:-1])

In [12]:
## loop through images to get embeddings

class_list = os.listdir('pytorch_images_tv/train')


In [13]:
print(class_list)

['homegarden', 'appliances', 'computers', 'office', 'sportsleisure', 'kidstoys', 'phones', 'healthbeauty', 'videogames', 'diytools', 'clothes', 'other', 'booksfilmmusicgames']


In [15]:
#####PROCESS EACH TRAINING IMAGE THROUGH THE MODEL AND WRITE EMBEDDINGS TO A DICTIONARY######

images_dir = 'pytorch_images_tv/train'
embeddings_dict = {}

for cat in class_list:
  for i in os.listdir(os.path.join(images_dir, cat)):
    image_fp = os.path.join(images_dir, cat, i)
    batch_embeddings = model(process_img(os.path.join(images_dir, cat, i)))
    batch_embeddings = batch_embeddings.view(batch_embeddings.size(0), -1)

    for emb, file_name in zip(batch_embeddings, image_fp):
        # Get the image ID from the file name (assuming it is the filename without extension)
        image_id = image_fp.split(".")[0]

        # Create a dictionary for each embedding entry
        embeddings_dict[image_id] = emb.tolist()



In [16]:
#### Write the embeddings dictionary to a JSON file
# Specify the path to your JSON file
json_file_path = "final_embeddings_sep.json"

# Open the file in write mode
with open(json_file_path, 'w') as file:
    # Write the dictionary to the JSON file
    json.dump(embeddings_dict, file)

In [17]:
## Write the json embeddings to a final
with open('final_embeddings_sep.json', "r") as json_file:
  data_dict = json.load(json_file)